<a href="https://colab.research.google.com/github/sac-1999/Pandas-Cheat-Sheet/blob/main/06_Imputer_sklearn_for_missing_records.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [163]:
import os
import tarfile
import urllib.request

In [164]:
house_url = 'https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/housing/housing.tgz'
HOUSING_PATH = os.path.join('datasets', 'housing')

In [165]:
def fetch_housing_data(url = house_url, housing_path = HOUSING_PATH):
  os.makedirs(housing_path , exist_ok = True)
  tgz_path = os.path.join(housing_path , 'housing.tgz')
  urllib.request.urlretrieve(house_url, tgz_path)
  housing_file = tarfile.open(tgz_path)
  housing_file.extractall(housing_path)
  housing_file.close()

In [166]:
fetch_housing_data()

In [167]:
import pandas as pd

In [168]:
def load_housing_csv(housing_path = HOUSING_PATH):
  csv_path = os.path.join(housing_path, 'housing.csv')
  return pd.read_csv(csv_path)

In [169]:
df = load_housing_csv()

In [170]:
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [171]:
df.ocean_proximity.value_counts()

<1H OCEAN     9136
INLAND        6551
NEAR OCEAN    2658
NEAR BAY      2290
ISLAND           5
Name: ocean_proximity, dtype: int64

In [172]:
import matplotlib.pyplot as plt

In [173]:
#df.hist(bins = 40,figsize=(20,15))

In [174]:
import numpy as np
np.random.permutation(100)

array([71, 61, 46, 81, 65, 98, 94, 33, 43, 62, 59, 91, 28, 63,  9, 73, 99,
       38, 79,  6, 27, 87, 19, 36, 78, 44, 93,  0, 56, 55, 17, 70, 64, 66,
       80, 60, 26, 96, 40, 92, 90, 53, 11, 29, 82, 50, 88, 77, 14, 16, 89,
       57, 30, 48, 52, 12, 20, 76, 15, 68, 25, 41, 54, 31,  4, 95, 67, 34,
       37,  3, 75, 83, 39, 49, 74, 18, 69, 23, 47, 84, 10,  1, 42, 58, 72,
       13, 97, 35, 51, 21, 45,  5, 22,  7, 85,  2, 32,  8, 86, 24])

In [175]:
df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


In [176]:
df['median_income_category'] = pd.cut(df['median_income'],
       bins= [0., 1.5, 3.0, 4.5, 6., np.inf],
       labels = [1,2,3,4,5])

In [177]:
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,median_income_category
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY,5
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY,5
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY,5
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY,4
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY,3


# Stratified K- Fold Split

In [178]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2, random_state = 0)
final = split.split(df,df['median_income_category'])

In [179]:
for i in final:
  print(i[0].shape, i[1].shape)
  strat_train_set = df.loc[i[0]]
  strat_test_set = df.loc[i[1]]


(16512,) (4128,)


In [180]:
new_df = strat_train_set.copy()

In [181]:
new_df.corr()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
longitude,1.000000,-0.924903,-0.108097,0.041547,0.065183,0.096933,0.051827,-0.013645,-0.043236
latitude,-0.924903,1.000000,0.010784,-0.034359,-0.064436,-0.106834,-0.069613,-0.081262,-0.145570
housing_median_age,-0.108097,0.010784,1.000000,-0.359036,-0.317509,-0.293215,-0.300756,-0.113458,0.107144
total_rooms,0.041547,-0.034359,-0.359036,1.000000,0.929552,0.853612,0.918026,0.196382,0.137469
total_bedrooms,0.065183,-0.064436,-0.317509,0.929552,1.000000,0.874315,0.980162,-0.009282,0.053544
population,0.096933,-0.106834,-0.293215,0.853612,0.874315,1.000000,0.903795,0.003431,-0.023797
households,0.051827,-0.069613,-0.300756,0.918026,0.980162,0.903795,1.000000,0.011840,0.069177
median_income,-0.013645,-0.081262,-0.113458,0.196382,-0.009282,0.003431,0.011840,1.000000,0.688883
median_house_value,-0.043236,-0.145570,0.107144,0.137469,0.053544,-0.023797,0.069177,0.688883,1.000000


# Imputer sklearn for missing recordds

In [182]:
housing_data = new_df.drop(['median_income_category', 'median_house_value'], axis = 1)
housing_label = new_df['median_house_value'].copy()

In [183]:
housing_data.isnull().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        168
population              0
households              0
median_income           0
ocean_proximity         0
dtype: int64

In [184]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy = 'median')

In [185]:
num_col = [i for i in housing_data.columns if housing_data[i].dtypes != 'object']

In [186]:
imputer.fit(housing_data[num_col])

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='median', verbose=0)

In [187]:
imputer.statistics_

array([-118.49  ,   34.25  ,   29.    , 2128.5   ,  436.    , 1167.    ,
        410.    ,    3.5341])

In [188]:
X = imputer.transform(housing_data[num_col])

In [189]:
X.shape

(16512, 8)